In [35]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import matplotlib
import string as string_library
import numpy as np
import isodate
import csv
import urltools
import urllib
from scipy.stats import mannwhitneyu
import re
import os.path
from langdetect import detect
import string
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import fastcluster
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances
from scipy.spatial.distance import squareform
import sys
from scipy.cluster.hierarchy import fcluster, dendrogram
from nltk.stem.porter import PorterStemmer
import warnings
warnings.filterwarnings('ignore')
sys.setrecursionlimit(100000)
matplotlib.style.use('ggplot')

## YouTube - Exploratory Analyses
### Loading the dataset
First, let us load the dataset from the database.

In [2]:
con = sqlite3.connect('./youtube.db')
video = pd.read_sql_query('''SELECT v.autoId as autoId, 
                                    v.id as id,
                                    v.categoryId as categoryId,
                                    v.channelId as channelId,
                                    v.publishedAt as publishedAt,
                                    v.title as title,
                                    v.description as description,
                                    v.viewCount as viewCount,
                                    v.likeCount as likeCount,
                                    v.dislikeCount as dislikeCount,
                                    v.favoriteCount as favoriteCount,
                                    v.commentCount as commentCount,
                                    v.duration as duration,
                                    v.defaultLanguage as defaultLanguage,
                                    c.title as channelTitle,
                                    c.description as channelDescription,
                                    c.publishedAt as channelPublishedAt,
                                    c.viewCount as channelViewCount,
                                    c.commentCount as channelCommentCount,
                                    c.subscriberCount as channelSubscriberCount,
                                    c.videoCount as channelVideoCount,
                                    c.country as channelCountry
                                    from video v left join channel c on v.channelId = c.id''', con)
url = pd.read_sql_query('SELECT * from urlNumber', con)
urlResolve = pd.read_sql_query('SELECT * from urlResolve', con)
category = pd.read_sql_query('SELECT * from category', con)
channel = pd.read_sql_query('SELECT * from channel', con)
captions = pd.read_sql_query('SELECT * FROM captions', con)

Next, let's examine the count of videos we collected. The tables are ``video``, ``url``, ``urlResolve``, and ``category``.

In [3]:
video.shape

(515999, 22)

In [4]:
list(video.columns.values)

['autoId',
 'id',
 'categoryId',
 'channelId',
 'publishedAt',
 'title',
 'description',
 'viewCount',
 'likeCount',
 'dislikeCount',
 'favoriteCount',
 'commentCount',
 'duration',
 'defaultLanguage',
 'channelTitle',
 'channelDescription',
 'channelPublishedAt',
 'channelViewCount',
 'channelCommentCount',
 'channelSubscriberCount',
 'channelVideoCount',
 'channelCountry']

In [5]:
url.shape

(405471, 4)

In [6]:
list(url.columns.values)

['autoId', 'videoId', 'url', 'lineNumber']

In [7]:
urlResolve.shape

(660009, 5)

In [8]:
list(urlResolve.columns.values)

['autoId', 'urlId', 'url', 'urlOrder', 'code']

In [9]:
category.shape

(32, 3)

In [10]:
list(category.columns.values)

['autoId', 'id', 'key']

### Examining the videos
Next, let's examine the categories of the videos.

In [11]:
video['duration'] = video['duration'].map(lambda x: isodate.parse_duration(x).total_seconds())
video['duration'].describe()

count    515999.000000
mean        445.870275
std        1201.335379
min           0.000000
25%          61.000000
50%         172.000000
75%         366.000000
max       86400.000000
Name: duration, dtype: float64

### Manipulating the URLs

Mark all the following patterns as affiliate links.

In [12]:
def check_affiliate_link(url):
    regex = re.compile("(ad.admitad.com/g/)|(ad.admitad.com/goto/)|(www.audible.com/ep/youtube\\?.*)|(performance.affiliaxe.com/.*\\?aff_id=)|(performance.affiliaxe.com/.*&aff_id=)|(s.aliexpress.com/.*\\?af=)|(s.aliexpress.com/.*&af=)|(amazon.com/.*\\?tag=)|(amazon.com/.*&tag=)|(amazon.de/.*\\?tag=)|(amazon.de/.*&tag=)|(amazon.it/.*\\?tag=)|(amazon.it/.*&tag=)|(amazon.in/.*\\?tag=)|(amazon.in/.*&tag=)|(amazon.fr/.*\\?tag=)|(amazon.fr/.*&tag=)|(primevideo.com/.*\\?ref=)|(primevideo.com/.*&ref=)|(itunes.apple.com/.*\\?at=)|(itunes.apple.com/.*&at=)|(apple.com/.*\\?afid=)|(apple.com/.*&afid=)|(affiliates.audiobooks.com/.*\\?a_aid=.*&a_bid=)|(affiliates.audiobooks.com/.*\\?a_bid=.*&a_aid=)|(affiliates.audiobooks.com/.*&a_bid=.*&a_aid=)|(avantlink.com/.*\\?pw=)|(avantlink.com/.*&pw=)")
    match = re.search(regex, url)
    if match:
        return True
    regex = "(secure.avangate.com/.*\\?affiliate=)|(secure.avangate.com/.*&affiliate=)|(awin1.com/.*\\?awinaffid=)|(awin1.com/.*&awinaffid=)|(ad.zanox.com/ppc^)|(zenaps.com/rclick.php\\?)|(banggood.com/.*\\?p=)|(banggood.com/.*&p=)|(bookdepository.com/.*\\?a_aid=)|(bookdepository.com/.*&a_aid=)|(booking.com/.*\\?aid=)|(booking.com/.*&aid=)|(hop.clickbank.net^)|(anrdoezrs.net/click-)|(cj.dotomi.com^)|(dpbolvw.net/click-)|(emjcd.com^)|(jdoqocy.com/click-)|(kqzyfj.com/click-)|(qksrv.net^)|(tkqlhce.com/click-)|(designmodo.com/\\?u=)|(rover.ebay.com/.*\\?campid=)|(rover.ebay.com/.*&campid=)|(audiojungle.net/.*\\?ref=)|(audiojungle.net/.*&ref=)|(codecanyon.net/.*\\?ref=)|(codecanyon.net/.*&ref=)(marketplace.envato.com/.*\\?ref=)|(marketplace.envato.com/.*&ref=)|(graphicriver.net/.*\\?ref=)|(graphicriver.net/.*&ref=)|(themeforest.net/.*\\?ref=)|(themeforest.net/.*&ref=)|(videohive.net/.*\\?ref=)|(videohive.net/.*&ref=)|(buyeasy.by/cashback/)|(buyeasy.by/redirect/)|(flipkart.com/.*\\?affid=)|(flipkart.com/.*&affid=)|(gtomegaracing.com/.*\\?tracking=)|(gtomegaracing.com/.*&tracking=)|(search.hotellook.com/.*\\?marker=)|(search.hotellook.com/.*&marker=)|(hotmart.net.br/.*\\?a=)|(hotmart.net.br/.*&a=)|(7eer.net/c/)|(evyy.net/c/)|(kontrolfreek.com/.*\\?a_aid=)|(kontrolfreek.com/.*&a_aid=)|(online.ladbrokes.com/promoRedirect\\?key=)|(online.ladbrokes.com/promoRedirect\\?.*&key=)|(makeupgeek.com/.*\\?acc=)|(makeupgeek.com/.*&acc=)|(gopjn.com/t/)|(pjatr.com/t/)|(pjtra.com/t/)|(pntra.com/t/)|(pntrac.com/t/)|(pntrs.com/t/)|(click.linksynergy.com/.*\\?id=)|(click.linksynergy.com/.*&id=)|(go.redirectingat.com/.*\\?id=)|(go.redirectingat.com/.*&id=)|(olymptrade.com/.*\\?affiliate_id=)|(olymptrade.com/.*&affiliate_id=)|(rstyle.me^)|(shopstyle.it^)|(shareasale.com/r.cfm^)|(shareasale.com/m-pr.cfm^)|(shareasale.com/u.cfm^)|(apessay.com/.*\\?rid=)|(apessay.com/.*&rid=)|(tatacliq.com/.*\\?cid=af:)|(tatacliq.com/.*&cid=af:)|(thermoworks.com/.*\\?tw=)|(thermoworks.com/.*&tw=)|(zaful.com/.*\\?lkid=)|(zaful.com/.*&lkid=)"
    match = re.search(regex, url)
    if match:
        return True
    return False

def urlUtmMatches(url):
    regex = re.compile("((utm_source=.*)|(utm_term=.*)|(utm_campaign=.*)|(utm_content=.*)|(utm_medium=.*)|(aff_id=.*)|(campaignid=.*)){1}")
    match = re.search(regex, url)
    if match:
        return True
    return False

In [13]:
for i, row in urlResolve.iterrows():
    utm = urlUtmMatches(row['url'])
    aff = check_affiliate_link(row['url'])
    urlResolve.at[i, 'utmLink'] = utm
    urlResolve.at[i, 'affiliateLink'] = aff
    urlResolve.at[i, 'anyAffLink'] = utm or aff

How did these resolved affiliate links vary by domain?

In [23]:
def get_information_on_column(column):
    print urlResolve[column].value_counts()
    urls = url[url['autoId'].isin(urlResolve[urlResolve[column] == True].urlId.tolist())]
    print str(urls.shape[0]) + " urls have " + column
    vids = video[video['id'].isin(urls['videoId'].tolist())].copy()
    print str(vids.shape[0]) + " videos have " + column
    print str(vids.shape[0]*1.0/video.shape[0] * 100) + "% of videos"

In [24]:
get_information_on_column('affiliateLink')

False    648894
True      11115
Name: affiliateLink, dtype: int64
7876 urls have affiliateLink
2891 videos have affiliateLink
0.560272403629% of videos


In [25]:
get_information_on_column('utmLink')

False    654937
True       5072
Name: utmLink, dtype: int64
3518 urls have utmLink
2204 videos have utmLink
0.427132610722% of videos


In [26]:
get_information_on_column('anyAffLink')

False    644095
True      15914
Name: anyAffLink, dtype: int64
10785 urls have anyAffLink
4721 videos have anyAffLink
0.914924253729% of videos


In [30]:
videos = []
with open("sentences_en1.tsv", 'rt') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    for row in reader:
        videos.append([row[0], row[1], row[2], row[3]])
        
videos_en_new = pd.DataFrame(videos, columns=['id', 'description', 'channelTitle', 'sentence'])
del videos

In [31]:
features = ["additional","also","auto","best","call","car","checkout","code","codes","county","design","discount","download","email","free","get","go","good","help","info","information","like","live","make","making","music","name","please","project","quality","sales","save","series","services","sponsors","stream","superior","tips","us","use","using","world","year"]
weights = [-0.000091225325396, -0.000154690952283, -0.044722324741698, -0.004190488434367, -0.000091225325396, -0.044722324741698, 1.865925657523899, 1.999712635243791, 1.000000000000000, -0.004190488434367, -0.100454270569854, 0.500000000000000, -0.000183391354017, -0.000091225325396, -0.000041538248269, -0.000082986726799, -0.015997331310574, -0.067165231081675, -0.000082986726799, -0.000091225325396, 0.133993499419597, -0.033289039488179, -0.133534800800062, -0.016534227402228, -0.004190488434367, 0.000000000000000, -0.118632763901686, -0.000091225325396, -0.067165231081675, -0.004190488434367, -0.044722324741698, 0.033589479820219, -0.000082986726799, -0.500000000000000, -0.000082986726799, 0.133507084780844, 0.133915682228483, -0.032865467396576, -0.000091225325396, 0.016179781961365, -0.033575751261456, -0.000418021836881, -0.000183391354017]

def get_vector(sentence):
    vector = [0] * len(features)
    tokens = tokenize(sentence)
    for token in tokens:
        if token in features:
            vector[features.index(token)] += 1
    return vector

In [36]:
def tokenize(line):
    if (line is None):
        line = ''
    printable = set(string_library.printable)
    line = ''.join(filter(lambda x: x in printable, line)) 
    stopwords = nltk.corpus.stopwords.words('english')

    tokenizer = nltk.RegexpTokenizer(r'[a-zA-Z]*\'[a-zA-Z]*|\w+')
    
    tokens = []
    
    line = re.sub(r'(http[s]?://|www.)(?:[a-zA-Z]|[0-9]|[$-_@.&+]*|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))*', '', line).lower()
    tokens.extend(tokenizer.tokenize(line))
    
    tokens_ = [f.strip(string_library.punctuation) for f in tokens]
    for i in range(len(tokens_)):
        if tokens_[i][0:2] == "b'":
            tokens_[i] = tokens[i][2:]
    tokens_ = [f for f in tokens_ if f != '' and f not in stopwords and len(f) != 1]
    tokens_ = [f for f in tokens_ if not (f.isdigit() or f[0] == '-' and f[1:].isdigit())]
    #tokens_ = [stemmer.stem(f) for f in tokens_]

    return tokens_

In [38]:
matches = {}
count = 0
for i, row in videos_en_new.iterrows():
    tot = 0
    vector = get_vector(row['sentence'])
    for j in range(len(weights)):
        tot += weights[j] * vector[j]
    if tot > 1:
        count +=1
        matches[row.id] = True

In [39]:
print "Number of coupon code disclosures found ",
print count

Number of coupon code disclosures found  2346


In [40]:
coupon_vids = matches.keys()
print "Number of videos with coupon codes ",
print len(coupon_vids)

Number of videos with coupon codes  1553


In [41]:
urls = url[url['autoId'].isin(urlResolve[urlResolve['anyAffLink'] == True].urlId.tolist())]
aff_vids = urls['videoId'].tolist()

In [45]:
for item in aff_vids:
    matches[item] = True

In [47]:
count = 0
for i, row in video.iterrows():
    if row.id in matches:
        count += 1

In [48]:
print "Number of videos with any affiliate link or coupon code ",
print count

Number of videos with any affiliate link or coupon code  6006


In [50]:
print str(count*1.0/video.shape[0]*100) + "%"

1.1639557441%


#### Affiliate video duration
How do the duration of the affiliate videos vary?

In [51]:
#affiliate_videos['duration'].describe()

In [52]:
# def get_language(x):
#     language = 'Unknown'
#     try:
#         language = detect(x.description.strip())
#     except:
#         pass
#     return language

In [53]:
# #used this to get videos for the study
# i = 0
# count = 0
# import random
# while i < affiliate_videos.shape[0]:
#     #random_index = random.randint(0, affiliate_videos.shape[0])
#     random_index = i
#     i+=1
#     vid = affiliate_videos.iloc[random_index]
#     if vid.duration > 300 or len(vid.description) > 5000 or vid.viewCount < 5000:
#         continue
# #     print "https://www.youtube.com/watch?v=" + str(vid.id)
# #     print vid.description[0:200]
# #     print "_______________________________________________________________________________"
#     count+=1
# print count